In [ ]:
!hostname

In [ ]:
import torch
# import torchvision.models as models
import torch.distributed as dist
# import torch.optim as optim
import scipy.stats as stats
from pathlib import Path
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from rich.progress import track


# # oialr_checkpoint_folder = "/hkfs/work/workspace/scratch/qv2382-madonna/madonna/models/svd-final/12/weights/epoch_299.pt"
# oialr_checkpoint_folder = "/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/svd-nodroppath/2/weights/epoch_299.pt"
# # baseline_checkpoint_folder = "/hkfs/work/workspace/scratch/qv2382-madonna/madonna/models/baseline-final/8/weights/epoch_299.pt"
# baseline_checkpoint_folder = "/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/baseline-nodroppath/1/weights/epoch_299.pt"

checkpoints = {}
# ortho-sigma
# /hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/42/weights3
# /hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/42/weights2
# /hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/43/weights1
# /hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/44/weights0
# checkpoints["ortho-sigma"] = {
#     0: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/44/weights0"),
#     1: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/43/weights1"),
#     2: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/42/weights2"),
#     3: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/42/weights3"),
# }

# checkpoints["rand-sigma"] = {
#     0: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug-randsigma/0/weights0"),
#     1: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug-randsigma/1/weights1"),
#     2: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug-randsigma/2/weights2"),
#     3: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug-randsigma/0/weights3"),
# }

# checkpoints_resnet18 = {}
# checkpoints_resnet18["ortho-sigma"] = {
#     3: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma/9/weights3"),
#     0: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma/10/weights0"),
#     1: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma/11/weights1"),
#     2: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma/12/weights2"),
# }

# checkpoints_resnet18["rand-sigma"] = {
#     3: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-randsigma/0/weights3"),
#     0: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-randsigma/1/weights0"),
#     2: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-randsigma/2/weights2"),
#     1: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-randsigma/3/weights1"),
# }
checkpoints["resnet"] = {
	0: Path("/hkfs/work/workspace/scratch/qv2382-madonna2/madonna/models/baseline/9/weights/"),
	1: Path("/hkfs/work/workspace/scratch/qv2382-madonna2/madonna/models/baseline/23/weights/"),
	2: Path("/hkfs/work/workspace/scratch/qv2382-madonna2/madonna/models/baseline/24/weights/"),
}
	
checkpoints["vit"] = {
	0: Path("/hkfs/work/workspace/scratch/qv2382-madonna2/madonna/models/baseline/28/weights/"),
	1: Path("/hkfs/work/workspace/scratch/qv2382-madonna2/madonna/models/baseline/29/weights/"),
	2: Path("/hkfs/work/workspace/scratch/qv2382-madonna2/madonna/models/baseline/31/weights/"),
}

# combos = [[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]
# epochs = ["000", "024", "049", "074", "099", "124", "149", "174", "199", "224", "249", "274", "299", ]  # 
epochs = [
    "epoch_000.pt", 
    'epoch_004.pt',  
    "epoch_009.pt", 
    'epoch_014.pt',  
    "epoch_019.pt", 
    'epoch_024.pt',
    'epoch_029.pt', 
    'epoch_034.pt',
    'epoch_039.pt',  
    'epoch_044.pt',
    'epoch_049.pt',  
    'epoch_054.pt',
    'epoch_059.pt',  
    'epoch_064.pt',
    'epoch_069.pt',  
    'epoch_074.pt',
    'epoch_079.pt',  
    'epoch_084.pt',
    'epoch_089.pt',  
    'epoch_094.pt',
    'epoch_099.pt',  
    'epoch_104.pt',
    'epoch_109.pt',  
    'epoch_114.pt',
    'epoch_119.pt',
    'epoch_124.pt',
]

# svd_model_dict = torch.load(oialr_checkpoint_folder, map_location="cpu")
# baseline_model_dict = torch.load(baseline_checkpoint_folder, map_location="cpu")
# print(list(baseline_model_dict["model"].keys()))

# x = torch.rand(10)
# print(x.nbytes)
# get 2D reps for a checkpoint (fn)

def get_2d_reps(model_dict, use_qr, float64=False):
    ret_dict = {}
    # nbytes = 0
    for n in model_dict['model']:
        p = model_dict['model'][n]
        # print(n, p.shape, p.nbytes)
        if p.ndim < 2:
            continue

        hld = p
        if p.ndim > 2:  # collapse down to 2D
            shp = p.shape
            hld = p.view(p.shape[0], -1)
        # print(hld.shape)
        trans = hld.shape[0] < hld.shape[1]
        if trans:  # make 2D rep TS
            hld = hld.T

        if float64:
            hld = hld.to(torch.float64)
        if not use_qr:
            u, s, vh = torch.linalg.svd(hld, full_matrices=False)
            ret_dict[n] = {'u': u, 's': s, 'vh': vh, 'trans': trans, "w": hld}
        else:
            q, r = torch.linalg.qr(hld, mode="complete")
            ret_dict[n] = {'q': q, "r": r, 'trans': trans, "w": hld}
        
        # nbytes += u.nbytes
        # nbytes += s.nbytes
        # nbytes += vh.nbytes
    # print(nbytes)
    return ret_dict

def get_2d_reps_from_names(model_dict, base_names, append_names, use_qr, float64=False):
    ret_dict = {}
    # nbytes = 0
    for base in base_names:
        p = model_dict['model'][base]
        # print(base, p.shape, p.nbytes)
        if p.ndim < 2:
            continue

        hld = p
        if p.ndim > 2:  # collapse down to 2D
            shp = p.shape
            hld = p.view(p.shape[0], -1)
        trans = hld.shape[0] < hld.shape[1]
        if trans:  # make 2D rep TS
            hld = hld.T

        for app in append_names[base]:
            app_p = model_dict['model'][app]
            # print(base, app, app_p.shape)
            same_dim = torch.nonzero(torch.tensor(hld.shape) == torch.tensor(app_p.shape)).flatten()[0].item()
            # since its the same dim, need to cat it on the OTHER dim...
            same_dim = (same_dim + 1) % 2
            # print(base, app, hld.shape, app_p.shape)
            hld = torch.cat([hld, app_p.unsqueeze(same_dim)], dim=same_dim)

        if float64:
            hld = hld.to(torch.float64)

        # print(base, hld.shape)
        if not use_qr:
            u, s, vh = torch.linalg.svd(hld, full_matrices=False)
            ret_dict[base] = {'u': u, 's': s, 'vh': vh, 'trans': trans, "w": hld}
        else:
            q, r = torch.linalg.qr(hld, mode="reduced")
            ret_dict[base] = {'q': q, "r": r, 'trans': trans, "w": hld}
    #     nbytes += u.nbytes
    #     nbytes += s.nbytes
    #     nbytes += vh.nbytes
    # # print(nbytes)
    return ret_dict

def get_1d_associated_weights(model_dict):
    wgts_to_join = {}
    nbytes = 0
    last_miltidim_w = None
    ignore_list = ["pos_embed", "cls_token"]
    for n in model_dict["model"]:
        p = model_dict["model"][n]
        # print(n, p.shape, p.nbytes)
        if n in ignore_list:
            continue
        if p.ndim > 1:  # and last_miltidim_w is None:
            # print(f'mutli dim, start: {n} shape: {p.shape}')
            last_miltidim_w = n
            if n in wgts_to_join:
                raise ValueError
            wgts_to_join[n] = []
        elif last_miltidim_w is not None and p.ndim == 1:
            wgts_to_join[last_miltidim_w].append(n)
            # print(f"to join with {last_miltidim_w}: {n}")

        # print(n, p.shape)
    return wgts_to_join

class ModelRepsGen(object):
    def __init__(self):
        pass

    @staticmethod
    def get_model_reps_gen(cat_1d: bool = False, use_qr: bool = False):
        for ranks in combos:  # combos are 2d tuples with rankA, rankB
            # getting results
            # print(f"working on combo: {ranks}")
            for epoch in epochs:  # load models for each epoch
                if not load_models:
                    model_a = torch.load(checkpoints["ortho-sigma"][ranks[0]] / f"epoch_{epoch}.pt", map_location=map_location)
                    model_b = torch.load(checkpoints["ortho-sigma"][ranks[1]] / f"epoch_{epoch}.pt", map_location=map_location)
                else:
                    model_a = models[ranks[0]][epoch]
                    model_b = models[ranks[1]][epoch]
                # get append lists:
                if cat_1d:
                    names = get_1d_associated_weights(model_a)
                    rep_dict_a = get_2d_reps_from_names(model_dict=model_a, base_names=names.keys(), append_names=names, use_qr=use_qr)
                    rep_dict_b = get_2d_reps_from_names(model_dict=model_b, base_names=names.keys(), append_names=names, use_qr=use_qr)
                else:
                    rep_dict_a = get_2d_reps(model_dict=model_a, use_qr=use_qr)
                    rep_dict_b = get_2d_reps(model_dict=model_b, use_qr=use_qr)
                yield (ranks, epoch, rep_dict_a, rep_dict_b)

    def __len__():
        return len(combos) + len(epochs)
    
gen_itter = ModelRepsGen()
# sim = cossim(buff[i], buff[j])
# map_location = "cpu"  #"gpu:0" if torch.cuda.is_available() else "cpu"
# chkpt = torch.load(checkpoints_resnet18["ortho-sigma"][0] / "epoch_299.pt", map_location=map_location)
# for n in chkpt["model"]:
#     p = chkpt["model"][n]
#     print(n, p.shape)
# _ = get_2d_reps(checkpoints["ortho-sigma"][0] / "epoch_299.pt")
# test = get_1d_associated_weights(model_dict=chkpt)
# reps_2d_app = get_2d_reps_from_names(model_dict=chkpt, base_names=test.keys(), append_names=test, filepath=checkpoints["ortho-sigma"][0] / "epoch_299.pt")


In [ ]:
!nvidia-smi

In [ ]:
load_models = False
models = {}
map_location = "cuda:0" if torch.cuda.is_available() else "cpu"
if load_models:
    for rank in range(4):
        models[rank] = {}
        for epoch in epochs:
            models[rank][epoch] = torch.load(checkpoints_resnet18["ortho-sigma"][rank] / f"epoch_{epoch}.pt", map_location=map_location)
            # models[rank][epoch] = torch.load(checkpoints["ortho-sigma"][rank] / f"epoch_{epoch}.pt", map_location=map_location)
        print(f"loaded models for rank: {rank}")

In [ ]:
# Original compare with 5 epochs previous
run_cell = False
if run_cell:
    avg_sim = {}
    avg_offdiag_sim = {}
    avg_norm = {}
    # load 1st model and get uvts
    model = torch.load(checkpoints["vit"][2] / f"{epochs[0]}", map_location=map_location)
    last_uvts = get_2d_reps(model, use_qr=False)
    # loop through next models
    epoch = 5
    for comp in epochs[1:]:
        print(f"comp_model: {checkpoints['vit'][2] / comp}")
        comp_model = torch.load(checkpoints["vit"][2] / comp, map_location=map_location)
        comp_uvts = get_2d_reps(comp_model, use_qr=False)
        avg_sim[epoch] = {}
        avg_offdiag_sim[epoch] = {}
        avg_norm[epoch] = {}
        for n in comp_uvts:
            uvh_p = last_uvts[n]['u'] @ last_uvts[n]["vh"]
            uvh_c = comp_uvts[n]['u'] @ comp_uvts[n]["vh"]
            sim = (uvh_p.T @ uvh_c) / (torch.max(torch.norm(uvh_p, dim=0)) * torch.max(torch.norm(uvh_c, dim=0)))
            avg_sim[epoch][n] = sim.diag().mean().item()
            sim = sim.fill_diagonal_(0)
            avg_offdiag_sim[epoch][n] = sim.sum().item()
            uvh_p = uvh_p.to("cpu")
            uvh_c = uvh_c.to("cpu")
            # norm = torch.linalg.norm(uvh_p @ uvh_p.T - uvh_c @ uvh_c.T)
            # avg_norm[epoch][n] = norm.item()

        last_uvts = comp_uvts
        epoch += 5


In [ ]:
run_cell = False  # QR comparison

model_to_check = "vit"

if run_cell:
    use_qr = True
    # Original compare with 5 epochs previous
    avg_sim = {}
    avg_offdiag_sim = {}
    avg_norm = {}
    r_dist = {}
    # load 1st model and get uvts
    model = torch.load(checkpoints[model_to_check][2] / f"{epochs[0]}", map_location=map_location)
    last_uvts = get_2d_reps(model, use_qr=use_qr)
    # loop through next models
    epoch = 5
    for comp in epochs[1:]:
        print(f"comp_model: {checkpoints[model_to_check][2] / comp}")
        comp_model = torch.load(checkpoints[model_to_check][2] / comp, map_location=map_location)
        comp_uvts = get_2d_reps(comp_model, use_qr=use_qr)
        avg_sim[epoch] = {}
        avg_offdiag_sim[epoch] = {}
        avg_norm[epoch] = {}
        r_dist[epoch] = {}
        for n in comp_uvts:
            q_p = last_uvts[n]['q']
            q_c = comp_uvts[n]['q']
            sim = (q_p.T @ q_c) / (torch.max(torch.norm(q_p, dim=0)) * torch.max(torch.norm(q_c, dim=0)))
            avg_sim[epoch][n] = sim.diag().mean().item()
            sim = sim.fill_diagonal_(0)
            avg_offdiag_sim[epoch][n] = sim.sum().item()

            rp = last_uvts[n]['r']
            rc = comp_uvts[n]['r']
            
            r_distn = torch.sqrt((rp - rc).pow(2).sum()).item()
            r_dist[epoch][n] = r_distn

        #     print(r_dist)
        #     break
        # break
            # uvh_p = uvh_p.to("cpu")
            # uvh_c = uvh_c.to("cpu")
            # norm = torch.linalg.norm(uvh_p @ uvh_p.T - uvh_c @ uvh_c.T)
            # avg_norm[epoch][n] = norm.item()


        last_uvts = comp_uvts
        epoch += 5


In [ ]:

# load R/Sim mats from csv
from pathlib import Path
import pandas as pd

model_target = "resnet"
heatmap_min = 0.8
line_min = 0.75

r_dist_df = pd.read_csv(f"{model_target}/qr/rdist.csv")
# r_sim_df = pd.read_csv(f"{model_target}/qr/rsim.csv")
sim_dist_df = pd.read_csv(f"{model_target}/qr/sim.csv")

r_dist_df = r_dist_df.set_index("Unnamed: 0")
# r_sim_df = r_sim_df.set_index("Unnamed: 0")
sim_dist_df = sim_dist_df.set_index("Unnamed: 0")
print(r_dist_df.index[:3])
# print()
# r_dist_df = 1 - r_dist_df.iloc[:, :-5] / r_dist_df.to_numpy().max()
r_dist_df = r_dist_df.iloc[:, :-5]

means = r_dist_df.iloc[3:].mean(0).values

import seaborn as sns
import matplotlib.pyplot as plt
# from matplotlib import rc
new_rc = {
    # "figure.figsize": figSize,
    "figure.autolayout": True,
    "text.usetex": True,
    "axes.labelsize": 10,
    "font.size": 8,
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8 ,
    'font.family': 'serif',
    'font.serif': ["Palatino"],
}
# rc(**new_rc)
plt.rcParams.update(new_rc)

# avg_sim_db = pd.DataFrame.from_dict(avg_sim)

# print(avg_sim_db)
fig, ((ax_heatmap, ax_colorbar), (ax_line, ax4)) = plt.subplots(
    2, 2, sharex="col",
    figsize=(10, 7),
    gridspec_kw={'height_ratios': [4, 1], "width_ratios": [15, 1], "hspace": 0.01, "wspace": 0.01}
)

sns.heatmap(r_dist_df, ax=ax_heatmap, cbar_ax=ax_colorbar, vmin=heatmap_min, vmax=1.0)
x = [c + 0.5 for c in range(len(means))]
sns.lineplot(x=x, y=means, color='black', ax=ax_line)
ax_line.set_ylim([line_min, 1.])
ax_line.set_yticks([0.8, 1.0])
ax_line.set_xlabel("Epoch", fontsize=20)
ax_line.tick_params(labelsize=14)

offset = 10 if model_target == "vit" else 40
ax_heatmap.set_yticks([offset, r_dist_df.shape[0] - offset])  # Ticks at the beginning and end
ax_heatmap.set_yticklabels(['Last Layers', 'First Layers'], rotation=90, fontsize=14)
ax_heatmap.tick_params(length=0)
ax_heatmap.set_ylabel("Network layer", fontsize=20)

ax_colorbar.set_ylabel("Similarity", fontsize=14, labelpad=6)
ax_colorbar.set_yticks([0.80, 0.85, 0.9, 0.95, 1.0])
ax_colorbar.set_yticklabels([f"{x:.2f}" for x in ax_colorbar.get_yticks()], size=14)

# ax_line.set_xticks(ax_heatmap.get_xticks())
# ax_line.set_xticklabels(ax_heatmap.get_xticks(), size=14)

ax_line.set_ylabel("Mean\nSimilarity", fontsize=14)
ax4.axis('off')

# Optional: Adjust layout
plt.tight_layout()
fig.savefig(f"/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/analysis/figures/{model_target}-r-cdist.pdf", dpi=1000)
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# from matplotlib import rc
new_rc = {
    # "figure.figsize": figSize,
    "figure.autolayout": True,
    "text.usetex": True,
    "axes.labelsize": 10,
    "font.size": 8,
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8 ,
    'font.family': 'serif',
    'font.serif': ["Palatino"],
}
# rc(**new_rc)
plt.rcParams.update(new_rc)

avg_sim_db = pd.DataFrame.from_dict(avg_sim)

# print(avg_sim_db)
fig, ((ax_heatmap, ax_colorbar), (ax_line, ax4)) = plt.subplots(
    2, 2, sharex="col",
    figsize=(8, 5),
    gridspec_kw={'height_ratios': [4, 1], "width_ratios": [15, 1], "hspace": 0}
)

sns.heatmap(avg_sim_db, vmin=0.75, vmax=1.0, ax=ax_heatmap, cbar_ax=ax_colorbar)
means = avg_sim_db.mean(0).values
x = [c + 0.5 for c in range(len(means))]
sns.lineplot(x=x, y=means, color='black', ax=ax_line)

# ax_line.set(ylim=(0.78, 1.0))

ax_heatmap.set_yticks([10, avg_sim_db.shape[0] - 10])  # Ticks at the beginning and end
ax_heatmap.set_yticklabels(['Last Layers', 'First Layers'], rotation=90, fontsize=14)
ax_heatmap.tick_params(length=0)
ax_heatmap.set_ylabel("Network layer", fontsize=20)

ax_colorbar.set_ylabel("Stability", fontsize=20)
# ax_colorbar.set_yticks([0.8, 0.85, 0.9, 0.95, 1.0])
ax_colorbar.set_yticklabels([f"{x:.2f}" for x in ax_colorbar.get_yticks()], size=14)

# ax_line.set_xticks(ax_heatmap.get_xticks())
# ax_line.set_xticklabels(ax_heatmap.get_xticks(), size=14)

ax_line.set_ylabel("Mean\nStability", fontsize=16)
ax4.axis('off')

# Optional: Adjust layout
plt.tight_layout()
fig.savefig("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/analysis/figures/vit_sim_plot.pdf", dpi=1000)
plt.show()


In [ ]:
avg_offsim_db = pd.DataFrame.from_dict(avg_offdiag_sim)
print(avg_offsim_db.shape)
sns.heatmap(avg_offdiag_sim)#, vmin=0.8, vmax=1.0)

In [ ]:
norm_db = pd.DataFrame.from_dict(avg_norm)
# print(norm_db)
# sns.heatmap(norm_db)#, vmin=0.8, vmax=1.0)

fig, ((ax_heatmap, ax_colorbar), (ax_line, ax4)) = plt.subplots(
    2, 2, sharex="col",
    figsize=(8, 5),
    gridspec_kw={'height_ratios': [4, 1], "width_ratios": [15, 1], "hspace": 0}
)

sns.heatmap(norm_db, ax=ax_heatmap, cbar_ax=ax_colorbar)
means = norm_db.mean(0).values
x = [c + 0.5 for c in range(len(means))]
sns.lineplot(x=x, y=means, color='black', ax=ax_line)
# ax_line.set(ylim=(0.78, 1.0))

ax_heatmap.set_yticks([10, norm_db.shape[0] - 10])  # Ticks at the beginning and end
ax_heatmap.set_yticklabels(['Last Layers', 'First Layers'], rotation=90, fontsize=12)
ax_heatmap.tick_params(length=0)
ax_heatmap.set_ylabel("Network layer", fontsize=18)

ax_colorbar.set_ylabel("Rotationally Invarient Metric", fontsize=18)
# ax_colorbar.set_yticks([0.8, 0.85, 0.9, 0.95, 1.0])
ax_colorbar.set_yticklabels([int(i) for i in ax_colorbar.get_yticks()], size=12)

ax_line.set_ylabel("Mean\nMetric", fontsize=16)
# ax_line.set_xticklabels(ax_line.get_xticks(), size=8)
ax4.axis('off')

# Optional: Adjust layout
plt.tight_layout()
fig.savefig("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/analysis/figures/vit_norm_plot.pdf", dpi=1000)
plt.show()

In [ ]:
# Goal here: compare basic 2D reps with each other (not including 1D appends)

# 1. load models for each epoch
# 2. get 2d reps
# 3. compare 2d reps (cossim) + record the averages (dict, layernames == keys)
# 4. make heatmap

avg_similarity = {}  # epoch: layer: 
for ranks, epoch, rep_dict_a, rep_dict_b in track(gen_itter.get_model_reps_gen(False, use_qr=False), description="Processing 2D w/o 1D layers",):
    # rep_dicts are [name] = {u, s, vh}
    if epoch not in avg_similarity:
        avg_similarity[epoch] = {}
    cc = 0
    for layer in rep_dict_a:  # keys are the same
        layer_a = rep_dict_a[layer]
        layer_b = rep_dict_b[layer]
        
        uvh_a = layer_a['u'] @ layer_a['vh']
        uvh_b = layer_b['u'] @ layer_b['vh']
        sim = (uvh_a.T @ uvh_b) / (torch.max(torch.norm(uvh_a, dim=0)) * torch.max(torch.norm(uvh_b, dim=0)))
        # qa = layer_a["q"]
        # qb = layer_b["q"]
        # sim = (qa.T @ qb) / (torch.max(torch.norm(qa, dim=0)) * torch.max(torch.norm(qb, dim=0)))
        sim = sim.diag()  # should be AtA because uvh is column vectors
        
        # print(layer, ranks, epoch, sim.mean())

        if layer in avg_similarity[epoch]:
            avg_similarity[epoch][layer] += sim.mean().item() / len(combos)
        else:  # NOTE: div by len(combos to get the average at the)
            avg_similarity[epoch][layer] = sim.mean().item() / len(combos)
    # print(uvh_a.T @ uvh_b)
print('done')

In [ ]:
# plotting heatmap from last step
import seaborn as sns

avg_sim = pd.DataFrame.from_dict(avg_similarity)
print(avg_sim)

sns.heatmap(avg_sim)#, vmin=0.8, vmax=1.0)

In [ ]:
# Goal here: compare 2D reps with each other (INCLUDING 1D appends)

# 1. load models
# 2. get append lists
# 3. get 2d reps
# 4. compare 2d reps (cossim) + record the averages (dict, layernames == keys)
# 5. make heatmap
avg_similarity_w1d = {}  # epoch: layer: 
c = 0
for ranks, epoch, rep_dict_a, rep_dict_b in track(gen_itter.get_model_reps_gen(True), description="Processing 2D with 1D layers",):
    if epoch not in avg_similarity_w1d:
        avg_similarity_w1d[epoch] = {}
    for layer in rep_dict_a:  # keys are the same
        layer_a = rep_dict_a[layer]
        layer_b = rep_dict_b[layer]
        uvh_a = layer_a['u'] @ layer_a['vh']
        uvh_b = layer_b['u'] @ layer_b['vh']
        sim = (uvh_a.T @ uvh_b) / (torch.max(torch.norm(uvh_a, dim=0)) * torch.max(torch.norm(uvh_b, dim=0)))
        sim = sim.diag()  # should be AtA because uvh is column vectors
        # print(layer, ranks, epoch, sim.mean())

        if layer in avg_similarity_w1d[epoch]:
            avg_similarity_w1d[epoch][layer] += sim.mean().item() / len(combos)
        else:  # NOTE: div by len(combos to get the average at the)
            avg_similarity_w1d[epoch][layer] = sim.mean().item() / len(combos)
    #     c += 1
    #     if c > 2:   
    #         break
    # break
print('done')

In [ ]:
# plotting heatmap from last step

avg_sim_w1d = pd.DataFrame.from_dict(avg_similarity_w1d)

# avg_sim_w1d /= 6

sns.heatmap(avg_sim_w1d, vmin=0.8, vmax=1.0)

In [ ]:
# Goal here: when 2D reps differ, do the sum of the matmul for that row == 1
avg_uvh_sums = {}  # epoch: layer: 
for ranks, epoch, rep_dict_a, rep_dict_b in gen_itter.get_model_reps_gen(False):
    # track(gen_itter.get_model_reps_gen(False), description="Processing 2D w/o 1D layers",):
    # print(ranks, epoch)
        # rep_dicts are [name] = {u, s, vh}
    if epoch not in avg_uvh_sums:
        avg_uvh_sums[epoch] = {}
    for layer in rep_dict_a:  # keys are the same
        layer_a = rep_dict_a[layer]
        layer_b = rep_dict_b[layer]
        uvh_a = layer_a['u'] @ layer_a['vh']
        uvh_b = layer_b['u'] @ layer_b['vh']
        sim = (uvh_a.T @ uvh_b) / (torch.max(torch.norm(uvh_a, dim=0)) * torch.max(torch.norm(uvh_b, dim=0)))
        # sim = sim.diag()  # should be AtA because uvh is column vectors
        # if epoch == "024":
        # sim[sim.abs() < sim[0, 0] * 0.001] = 0
        
        # print(sim[:5, :5], sim.abs().sum(1))
        # break
        
        if layer in avg_uvh_sums[epoch]:
            avg_uvh_sums[epoch][layer] += sim.sum().item() / sim.shape[0] / len(combos)
        else:  # NOTE: div by len(combos to get the average at the)
            avg_uvh_sums[epoch][layer] = sim.sum().item() / sim.shape[0] / len(combos)

    # layer = 'layer3.0.conv2.weight'
    # layer_a = rep_dict_a[layer]
    # layer_b = rep_dict_b[layer]
    # uvh_a = layer_a['u'] @ layer_a['vh']
    # uvh_b = layer_b['u'] @ layer_b['vh']
    # sim = (uvh_a.T @ uvh_b) / (torch.max(torch.norm(uvh_a, dim=0)) * torch.max(torch.norm(uvh_b, dim=0)))
    # # sim = sim.diag()  # should be AtA because uvh is column vectors
    # # if epoch == "024":
    # # sim[sim.abs() < sim[0, 0] * 0.001] = 0
    
    # print(sim[:, 0].sum().item(), sim.sum().item(), uvh_a.shape)
print('done')

In [ ]:
avg_sum = pd.DataFrame.from_dict(avg_uvh_sums)

# avg_sum_w1d /= 6
print(avg_sum)

sns.heatmap(avg_sum, vmin=0.8, vmax=1.0)

In [ ]:
# NOTE: this is with the 1d vecs folder in

# Goal here: when 2D reps differ, do the sum of the matmul for that row == 1
avg_uvh_sums_w1d = {}  # epoch: layer: 
c = 0
model_app_list = None
for ranks, epoch, rep_dict_a, rep_dict_b in track(gen_itter.get_model_reps_gen(cat_1d=True), description="Processing 2D with 1D layers",):
    if epoch not in avg_uvh_sums_w1d:
        avg_uvh_sums_w1d[epoch] = {}
    for layer in rep_dict_a:  # keys are the same
        layer_a = rep_dict_a[layer]
        layer_b = rep_dict_b[layer]
        uvh_a = layer_a['u'] @ layer_a['vh']
        uvh_b = layer_b['u'] @ layer_b['vh']
        sim = (uvh_a.T @ uvh_b) / (torch.max(torch.norm(uvh_a, dim=0)) * torch.max(torch.norm(uvh_b, dim=0)))
        # sim = sim.diag()  # should be AtA because uvh is column vectors
        
        if layer in avg_uvh_sums_w1d[epoch]:
            avg_uvh_sums_w1d[epoch][layer] += sim.abs().sum(1).mean().item() / len(combos)
        else:  # NOTE: div by len(combos to get the average at the)
            avg_uvh_sums_w1d[epoch][layer] = sim.abs().sum(1).mean().item() / len(combos)

In [ ]:
avg_sum_w1d = pd.DataFrame.from_dict(avg_uvh_sums_w1d)

# avg_sum_w1d /= 6
print(avg_sum_w1d)

sns.heatmap(avg_sum_w1d, vmin=0.8, vmax=1.0)

In [ ]:
# Testing for new ideas here:

# current idea: concat [u1, u2] and [v1, v2] to make a wider effect area, then take svd and compare us with each other

# for ranks in combos:  # combos are 2d tuples with rankA, rankB
#     # getting results
#     # print(f"working on combo: {ranks}")
#     for epoch in epochs:  # load models for each epoch
ranks = combos[0]
model_a = models[0][epochs[-1]]
# print(ranks)
model_b = models[1][epochs[-1]]

rep_dict_a = get_2d_reps(model_dict=model_a, use_qr=False, float64=True)
rep_dict_b = get_2d_reps(model_dict=model_b, use_qr=False, float64=True)

u1 = rep_dict_a['layer1.1.conv1.weight']["u"]
u2 = rep_dict_b['layer1.1.conv1.weight']["u"]
s1 = rep_dict_a['layer1.1.conv1.weight']["s"]
s2 = rep_dict_b['layer1.1.conv1.weight']["s"]
vh1 = rep_dict_a['layer1.1.conv1.weight']["vh"]
vh2 = rep_dict_b['layer1.1.conv1.weight']["vh"]
# u1 = rep_dict_a['conv1.weight']["u"]
# u2 = rep_dict_b['conv1.weight']["u"]
# s1 = rep_dict_a['conv1.weight']["s"]
# s2 = rep_dict_b['conv1.weight']["s"]
# vh1 = rep_dict_a['conv1.weight']["vh"]
# vh2 = rep_dict_b['conv1.weight']["vh"]
s_mean = (s1 + s2) / 2.

w1 = u1 @ torch.diag(s1) @ vh1
w2 = u2 @ torch.diag(s2) @ vh2

# print((u1.T @ u2).diag())

In [ ]:
u1f, s1f, vh1f = torch.linalg.svd(w1, full_matrices=True)
u2f, s2f, vh2f = torch.linalg.svd(w2, full_matrices=True)

q1r, r1r = torch.linalg.qr(w1, mode="reduced")
q2r, r2r = torch.linalg.qr(w2, mode="reduced")

q1f, r1f = torch.linalg.qr(w1, mode="complete")
q2f, r2f = torch.linalg.qr(w2, mode="complete")

print(u1f.shape, s1f.shape, vh1f.shape)
print(u1.shape, s1.shape, vh1.shape)

# print(u1.T @ u1f[:, :256])

In [ ]:
a = u1 @ vh1
b = u2 @ vh2

# a = q1f
# b = q2f

def get_rotation_btw_bases(basis_a, basis_b, halfway=True, print_angle=False):
    # TODO: rename basis_a -> to_rotate and basis_b -> target_basis
    # create a rotation to rotate basis_a to basis_b
    # returns: R so that R @ A == B

    # steps: 
    #   1. create rotation to rotate A all the way to B
    #   2. find angle of rotation
    #   3. modify rotation matrix to only rotation halfway between A and B

    # step 1: create rotation to rotate A all the way to B
    rot = torch.zeros((basis_a.shape[0], basis_a.shape[0]), dtype=basis_a.dtype, device=basis_a.device)
    # rot = a @ b.T
    for col in range(basis_a.shape[1]):
        rot += torch.kron(basis_a[:, col], basis_b[:, col].unsqueeze(1))
    #     # rot += basis_a[:, col].unsqueeze(1) @ basis_b[:, col].unsqueeze(0)
        
    # print(torch.allclose(rot, basis_a @ basis_b.T))
    # print()
    # if not halfway:
    #     return rot
    
    # step 2: find angle of rotation
    
    # angle = torch.arccos(0.5 * (torch.trace(rot) - 1))
    # print(f"\t {torch.trace(rot):.4f} \t{torch.rad2deg(angle).item():.4f}\t {(rot.T @ rot).diag().mean():.3f}")
    # print(f"\t {torch.linalg.eigvals(rot).real}")
    
    # if torch.isnan(angle):
    #     raise RuntimeError("nan angle")
    
    # if print_angle:
    #     print(f"Angle between bases: {torch.rad2deg(angle).item():.4f}")
    if not halfway:
        return rot

    # step 3. modify rotation matrix to only rotation halfway between A and B
    alpha = (2 * torch.cos(angle / 2) + 1) / rot.diag().sum()
    new_diag = alpha / rot.shape[0] * rot.diag()
    rot = rot.fill_diagonal_(0)
    rot += torch.diag(new_diag)

    return rot

print("U")
r = get_rotation_btw_bases(basis_a=u1f, basis_b=u2f, halfway=False, print_angle=True)
print("Vh")
r = get_rotation_btw_bases(basis_a=vh1f, basis_b=vh2f, halfway=False, print_angle=True)

# l = b @ a.T - a @ b.T
# r = b @ a.T

# print(r)

# ub, sb, vhb = torch.linalg.svd(b, full_matrices=False)
# b_inv = vhb.T @ (1/sb).diag() @ ub.T
# r = a @ b_inv
print(torch.allclose(r @ vh1, vh2))
# print(torch.linalg.det(u1f), torch.linalg.det(vh1))

# print(f"is R orthogonal?-> {torch.allclose(r.T, torch.linalg.inv(r))}")
# print(f"det of R: {torch.linalg.det(r)}")

# print(((r @ q1f) - q2f).max())
# # print(((r @ a) - b).mean())

In [ ]:
# rotate all of the bases to rank zero's bases
# get_model_reps_with_r0(cat_1d: bool = False, use_qr: bool = False, which_epoch="299"):
r0combos = [1, 2, 3]
cat_1d = True
use_qr = False
which_epoch = "299"
float64 = True

del model_0, rep_dict_0, rep_dict_l

if not load_models:
    model_0 = torch.load(checkpoints["ortho-sigma"][0] / f"epoch_{which_epoch}.pt", map_location=map_location)
else:
    model_0 = models[0][which_epoch]

if cat_1d:
    names = get_1d_associated_weights(model_a)
    rep_dict_0 = get_2d_reps_from_names(model_dict=model_0, base_names=names.keys(), append_names=names, use_qr=use_qr, float64=float64)
else:
    rep_dict_0 = get_2d_reps(model_dict=model_0, use_qr=use_qr, float64=float64)

# get full Us
for layer in rep_dict_0:
    layer_0 = rep_dict_0[layer]
    uf, _, vhf = torch.linalg.svd(layer_0['w'], full_matrices=True)
    layer_0["uf"] = uf
    layer_0["vhf"] = vhf

for partner in r0combos:  # combos are 2d tuples with rankA, rankB
    # getting results
    if not load_models:
        model_l = torch.load(checkpoints["ortho-sigma"][partner] / f"epoch_{which_epoch}.pt", map_location=map_location)
    else:
        model_l = models[partner][which_epoch]
    # get append lists:
    if cat_1d:
        rep_dict_l = get_2d_reps_from_names(model_dict=model_l, base_names=names.keys(), append_names=names, use_qr=use_qr, float64=float64)
    else:
        rep_dict_l = get_2d_reps(model_dict=model_l, use_qr=use_qr, float64=float64)

    # loop through the dict and move the 
    for layer in rep_dict_l:  # keys are the same
        layer_0 = rep_dict_0[layer]
        layer_l = rep_dict_l[layer]
        ufl, _, vhf = torch.linalg.svd(layer_l['w'], full_matrices=True)

        u0 = layer_0['u']
        uf0 = layer_0['uf']
        ul = layer_l['u']
        
        v0 = layer_0['vh'].T
        vf0 = layer_0['vhf'].T
        vl = layer_l['vh'].T
        vfl = vhf.T

        # print(partner, layer,)
        # get number target vectors (shapecount number of vectors with a max of 0
        # get number of vectors included in v0 (dont care about full, since there are not sigma values for those)
        
        # print(f"U: {(u0.T @ ul).max(dim=0)[0].max():.4f} \t full: {(uf0.T @ ufl).max(dim=0)[0].max():.4f}")
        ru = get_rotation_btw_bases(basis_a=u0.T, basis_b=ul.T, halfway=False, print_angle=False)

        # print(ru)
        # break

        # print(f"V: {(v0.T @ vl).max(dim=0)[0].max():.4f} \t full: {(vf0.T @ vlf).max(dim=0)[0].max():.4f}")
        rv = get_rotation_btw_bases(basis_a=v0.T, basis_b=vl.T, halfway=False, print_angle=False).T
        if layer_0['s'].ndim == 1:
            layer_0['s'] = layer_0['s'].diag()
        layer_0['s'] += ru.T @ layer_l['s'] @ rv  # the same as ru @ ul @ sigmal @ vl.T @ rvt

    # break

for layer in rep_dict_0:  # finish average of weights
    
    u0, vh0 = rep_dict_0[layer]['u'], rep_dict_0[layer]['vh']
    sig = rep_dict_0[layer]['s'] / len(r0combos) + 1
    rep_dict_0[layer]['w'] = u0

    
# create network
# validate on validation set
# create network average + validate with that

In [ ]:
# rep_dict_0 has the dictionary of the 2d representations

In [ ]:


wavg = (w1 + w2) / 2
ua, sa, vha = torch.linalg.svd(wavg, full_matrices=False)

uvh_1 = u1 @ torch.eye(u1.shape[1], dtype=u1.dtype, device=u1.device) @ vh1
uvh_2 = u2 @ torch.eye(u1.shape[1], dtype=u1.dtype, device=u1.device) @ vh2

uvh_avg = ua @ torch.eye(u1.shape[1], dtype=u1.dtype, device=u1.device) @ vha
uvh_avg2 = (uvh_1 + uvh_2) / 2.

# print(uvh_1[0])

uvh_1n = uvh_1 / torch.norm(uvh_1, dim=0)
# print(torch.norm(uvh_1, dim=0))
uvh_2n = uvh_2 / torch.norm(uvh_2, dim=0)

sim = (uvh_2n.T @ uvh_1n) # / (torch.max(torch.norm(uvh_2, dim=0)) * torch.max(torch.norm(uvh_1, dim=0)))
sim2 = (uvh_2.T @ uvh_1) #/ (torch.max(torch.norm(uvh_2, dim=0)) * torch.max(torch.norm(uvh_1, dim=0)))

sim_avg = (uvh_2.T @ uvh_avg)
sim_avg2 = (uvh_2.T @ uvh_avg2)
print(sim_avg.diag()[:8].tolist())
print(sim_avg2.diag()[:8].tolist())
print(sim2.diag()[:8].tolist())
print(1 - ((1 - sim2.diag()[:8]) / 2.))
# print(sim2.diag().min(), sim2.diag().max(), sim2.diag().std())

print('\n\n')

# print(torch.count_nonzero(sim.abs() >= 1e-2), sim.shape[0], sim.numel(), torch.count_nonzero(sim.abs() >= 1e-2) / sim.numel())
vals, inds = torch.topk(sim2.abs(), k=10, dim=1)

# print(inds[:4])
# print(inds[:4].sum(1))
# print(sim[sim < vals[:, -1].unsqueeze(1).expand(sim.shape)])
sim2[sim2 < vals[:, -1]] *= 0
# print(sim[:4, :4])


# torch.linalg.det(uvh_1)

# r = vh2 @ u1

# sim_new = (r @ uvh_1).T @ uvh_avg
# print(sim_new.diag()[:8])